```
Gautam Ahuja
Introduction to Machine Learning
Raghavendra Singh
Assignment 3 Code & Report
Implement PCA using Eigen library
```

Implement  PCA in python (10 marks) using https://github.com/stack-of-tasks/eigenpy

Generate random datasets with (m, n ) = [(100,100), (1000,1000), (10000, 10000), (10000,50000), (50000,50000)]
and do PCA with number of components = 0.1*n

Report run time of your implementation and compare with runtime of sklearn.decomposition.pca


Submit code and method for generating dataset along with above report.


In [1]:
# from eigenpy import quaternion
from sklearn.datasets import make_classification
import numpy as np
import eigenpy
import time

### Principal Component Analysis (PCA)
To perform PCA, we need to perform the following steps:
1. Standardize the data. Mean of each feature should be 0 and standard deviation should be 1. The standardization is done along each row.
2. Calculate the covariance matrix of the standardized data.
3. Calculate the eigenvalues and eigenvectors of the covariance matrix.
4. Sort eigenvalues in descending order and choose the top k eigenvectors that correspond to the k largest eigenvalues. k is the number of dimensions of the new feature subspace (k<=n).
5. Construct the projection matrix W from the selected k eigenvectors.
6. Transform the original dataset X via W to obtain a k-dimensional feature subspace Y.

We will use the eigenpy library to perform the PCA from scratch. The eigenpy library is a python wrapper for the Eigen library. The Eigen library is a C++ template library for linear algebra: matrices, vectors, numerical solvers, and related algorithms. It supports all matrix sizes, from small fixed-size matrices to arbitrarily large dense matrices, and even sparse matrices.

### Data Generation
We generate the datasets using the make_classification function from sklearn -- this creates mulclass dataset by allocating each classs one or more normally distributed clusters of points

In [3]:
# Generate random datasets with (m, n ) = [(100,100), (1000,1000), (10000, 10000), (10000,50000), (50000,50000)]
dataset = []
np.random.seed(42)
m = [100, 1000, 10000, 10000, 50000]
n = [100, 1000, 10000, 50000, 50000]
for i in range(len(m)):
    # dataset.append(np.random.rand(m[i], n[i]))
    dataset.append(make_classification(n_samples=m[i], n_features=n[i], random_state=42))
    print("Dataset shape: ", dataset[i].shape)

Dataset shape:  (100, 100)
Dataset shape:  (1000, 1000)
Dataset shape:  (10000, 10000)
Dataset shape:  (10000, 50000)
Dataset shape:  (50000, 50000)


#### Note
The (10000, 50000) and (50000, 50000) dataset is too large -- 18 GB -- and cannot be allocated in memory

It gives the following error:

```MemoryError: Unable to allocate 18.6 GiB for an array with shape (50000, 50000) and data type float64```

In [4]:
# Perform PCA with number of components = 0.1*n
n_components = [int(0.1 * n[i]) for i in range(len(n))]

The eigenpy has the following functions available:
1. AngleAxis: Represents a rotation by an angle around an axis.
2. ComputationInfo: Enumerates the possible outcomes of a linear algebra computation.
3. DecompositionOptions: Enumerates the possible options for a decomposition.
4. EigenSolver: Computes eigenvalues and eigenvectors of a matrix.
5. Exception: Base class for all exceptions thrown by EigenPy.
6. LDLT: Computes the LDLT decomposition of a matrix with partial pivoting.
7. LLT: Computes the LLT decomposition of a matrix.
8. MINRES: Computes the MINRES decomposition of a matrix.
9. Quaternion: Represents a rotation by a quaternion.
10. SelfAdjointEigenSolver: Computes eigenvalues and eigenvectors of a selfadjoint matrix.
11. SimdInstructionSetsInUse: Enumerates the SIMD instruction sets in use.
12. StdVec_MatrixXd: A std::vector of Eigen::MatrixXd.
13. StdVec_MatrixXi: A std::vector of Eigen::MatrixXi.
14. StdVec_VectorXd: A std::vector of Eigen::VectorXd.
15. StdVec_VectorXi: A std::vector of Eigen::VectorXi.
16. checkVersionAtLeast: Checks that the version of EigenPy is at least the given version.
17. fromEulerAngles: Constructs a rotation from Euler angles.
18. is_approx: Returns true if the two values are approximately equal.
19. seed: Seeds the random number generator.
20. sharedMemory: Returns the shared memory segment used by EigenPy.
21. solvers: Returns the list of available solvers.
22. toEulerAngles: Returns the Euler angles of a rotation.

In [5]:
def pca(dataset, n_components):
    # Standardize the dataset along rows
    dataset = (dataset - np.mean(dataset, axis=0))# / np.std(dataset, axis=0)
    # Compute the covariance matrix
    cov = np.cov(dataset.T)
    # Compute the eigenvalues and eigenvectors of the covariance matrix
    eig_vals = eigenpy.EigenSolver(cov).eigenvalues()
    eig_vecs = eigenpy.EigenSolver(cov).eigenvectors()
    
    # Sort the eigenvalues in descending order
    idx = eig_vals.argsort()[::-1]
    
    # idx = np.arange(0,len(eig_vals), 1)
    # idx = ([x for _,x in sorted(zip(eig_vals, idx))])[::-1]
    # Sort the eigenvectors according to the sorted eigenvalues
    eig_vals = eig_vals[idx]
    eig_vecs = eig_vecs[:, idx]
    # Compute the projection matrix
    proj_mat = eig_vecs[:, :n_components]
    # Project the dataset onto the projection matrix
    dataset = np.dot(dataset, proj_mat)
    return dataset


In [6]:
start_time = time.time()
for i in range(2):#len(dataset)):
    print("Dataset shape: ", dataset[i].shape)
    curr_time = time.time()
    reduced_dataset =  pca(dataset[i], n_components[i])
    print("Reduced dataset shape: ", reduced_dataset.shape)
    print("Time taken: ", time.time() - curr_time)
    print("--------------------------------------------------")
print("Total time taken: ", time.time() - start_time)

Dataset shape:  (100, 100)
Reduced dataset shape:  (100, 10)
Time taken:  0.019929885864257812
--------------------------------------------------
Dataset shape:  (1000, 1000)
Reduced dataset shape:  (1000, 100)
Time taken:  6.69486665725708
--------------------------------------------------
Total time taken:  6.715179443359375


### PCA Using Sklearn
Now we implement PCA using the sklearn library and note the time taken for the same.

In [7]:
from sklearn.decomposition import PCA
import time

In [8]:
start_time = time.time()
for i in range(2):#len(dataset)):
    print("Dataset shape: ", dataset[i].shape)
    curr_time = time.time()
    pca = PCA(n_components=n_components[i])
    reduced_dataset = pca.fit_transform(dataset[i])
    print("Reduced dataset shape: ", reduced_dataset[i].shape)
    print("Time taken: ", time.time() - curr_time)
    print("--------------------------------------------------")
print("Total time taken: ", time.time() - start_time)

Dataset shape:  (100, 100)
Reduced dataset shape:  (10,)
Time taken:  0.0053288936614990234
--------------------------------------------------
Dataset shape:  (1000, 1000)
Reduced dataset shape:  (100,)
Time taken:  1.2234978675842285
--------------------------------------------------
Total time taken:  1.229102373123169


## Time Measurement
We use the time library to measure the time taken for the PCA implementation using eigenpy and sklearn.

### Using eigenpy
Time taken for:
1. 100 x 100 dataset: 0.019929885864257812 seconds
2. 1000 x 1000 dataset: 6.69486665725708 seconds

### Using sklearn
Time taken for:
1. 100 x 100 dataset: 0.0053288936614990234 seconds
2. 1000 x 1000 dataset: 1.2234978675842285 seconds

## Note
I could not run the code for the (10000, 50000) and (50000, 50000) dataset due to memory constraints. I have run the code for the other datasets and have included the results in the report.

I could not run the PCA on (10000, 10000), (10000, 50000) and (50000, 50000) datasets due to memory and CPU constraints. I have run the code for the other datasets and have included the results in the report. Running on these always resulted in a Kernel Crash. My task manager showed that the memory usage was 100% and the CPU usage was 100% as well. I tried running the code on Google Colab as well, but the eigenpy library was not available there as it is only available for conda environments.